In [159]:
import numpy as np
import os
import sklearn.linear_model
import sklearn.metrics
import optuna
import sklearn.ensemble
import sklearn.pipeline
import sklearn.preprocessing
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pandas as pd
from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

### Loading Data

In [160]:
PATH_DATA = "data/"
X_train = np.load(os.path.join(PATH_DATA, "X_train.npy"))
X_test = np.load(os.path.join(PATH_DATA, "X_test.npy"))
y_train = np.load(os.path.join(PATH_DATA, "y_train.npy"))
y_test = np.load(os.path.join(PATH_DATA, "y_test.npy"))

X_train.shape, y_test.shape, X_test.shape, y_test.shape

((500, 30), (500,), (500, 30), (500,))

### Visualizing Data

### Model

In [161]:
svm_classifier = SVC(kernel='poly')
svm_classifier.fit(X_train, y_train)
y_pred = svm_classifier.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.74      0.80       201
           1       0.84      0.93      0.88       299

    accuracy                           0.85       500
   macro avg       0.85      0.83      0.84       500
weighted avg       0.85      0.85      0.85       500



In [162]:
def objective_SVM(trial: optuna.trial.Trial):
    t_C = trial.suggest_float("C", 0.1, 1, step=0.01)
    t_gamma = trial.suggest_categorical("gamma", ["scale", "auto"])
    t_kernel = trial.suggest_categorical("kernel", [ "poly", "rbf", "sigmoid"])

    svm_classifier = SVC(
        C=t_C,
        gamma=t_gamma,
        kernel=t_kernel,
    ) 
    
    svm_classifier.fit(X_train, y_train)
    y_pred = svm_classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

In [163]:
storage_name = "classification.db"
if os.path.exists(storage_name):
    os.remove(storage_name)

optuna.logging.set_verbosity(optuna.logging.WARNING)

study = optuna.create_study(
        storage=f"sqlite:///{storage_name}",
        study_name="Classification",
        load_if_exists=False,
        direction="maximize",
)

study.optimize(func=objective_SVM, n_trials=100)

In [164]:
print(f"Best obtrained accuracy: {study.best_trial.values}\nWith the following parameters: {study.best_trial.params}" )

Best obtrained accuracy: [0.984]
With the following parameters: {'C': 0.87, 'gamma': 'auto', 'kernel': 'poly'}


standardaizing doesn't imrpove, choice of kernel important, regularization and game very important explain why: 